In [2]:
import sys
sys.path.append('..')

from warnings import filterwarnings
filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [6]:
link_data_training = '../data/raw/Train/Admiralty/Data_Gabungan_Lainnya_2009.csv'
contoh = pd.read_csv(link_data_training)
contoh.head()

,Date,Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2009-01-01,,,,,,,,
1,2009-01-02,,,,,,,,
2,2009-01-03,,,,,,,,
3,2009-01-04,,,,,,,,
4,2009-01-05,,,,,,,,


In [3]:
import os
import glob
import re

# Path
base_train_path = '../data/raw/Train'
output_path = '../data/merged/'

# Cleaning Column Name
def clean_column_names(df):
    """
    Membersihkan dan menyeragamkan nama kolom pada DataFrame.
    """
    new_columns = []
    for col in df.columns:
        cleaned_col = col.strip().replace(' ', '_').lower()
        cleaned_col = re.sub(r'[().%°]', '', cleaned_col)
        new_columns.append(cleaned_col)
    df.columns = new_columns
    return df

# Merge Dataset Each City
def proses_semua_lokasi(path_induk, path_keluaran):
    """
    Melakukan loop ke semua folder, menggabungkan CSV, membersihkan nama kolom,
    dan menyimpan hasilnya.
    """
    print("Memulai proses otomatis untuk semua lokasi...")
    os.makedirs(path_keluaran, exist_ok=True)
    print(f"Folder output dipastikan ada di: {path_keluaran}")

    try:
        folder_lokasi = [f.name for f in os.scandir(path_induk) if f.is_dir()]
        print(f"Ditemukan {len(folder_lokasi)} folder lokasi untuk diproses.")
    except FileNotFoundError:
        print(f"❌ Error: Direktori induk tidak ditemukan di '{path_induk}'. Hentikan proses.")
        return

    for nama_lokasi in folder_lokasi:
        print(f"\n-> Memproses: '{nama_lokasi}'...")

        path_lokasi_spesifik = os.path.join(path_induk, nama_lokasi)
        daftar_file_csv = sorted(glob.glob(os.path.join(path_lokasi_spesifik, "*.csv")))

        if not daftar_file_csv:
            print(f"  -- Lewati '{nama_lokasi}', tidak ada file .csv ditemukan.")
            continue

        # PENTING: Lakukan pembersihan nama kolom SEBELUM digabung
        list_of_dfs = []
        for file in daftar_file_csv:
            df = pd.read_csv(file)
            # Langsung bersihkan nama kolom setelah file dibaca
            df = clean_column_names(df)
            list_of_dfs.append(df)

        # Gabungkan DataFrame yang nama kolomnya sudah seragam
        df_gabungan = pd.concat(list_of_dfs, ignore_index=True)

        # Cek jumlah kolom setelah digabung
        print(f"  -> Jumlah kolom setelah digabung: {len(df_gabungan.columns)}")

        nama_file_output = f"{nama_lokasi}_merged.csv"
        path_file_output = os.path.join(path_keluaran, nama_file_output)

        df_gabungan.to_csv(path_file_output, index=False)
        print(f"  ✅ Berhasil! Data dengan kolom bersih disimpan di: '{nama_file_output}'")

    print("\n\n🎉 Semua proses selesai!")

# Running
proses_semua_lokasi(base_train_path, output_path)

Memulai proses otomatis untuk semua lokasi...
Folder output dipastikan ada di: ../data/merged/
Ditemukan 44 folder lokasi untuk diproses.

-> Memproses: 'Admiralty'...
  -> Jumlah kolom setelah digabung: 9
  ✅ Berhasil! Data dengan kolom bersih disimpan di: 'Admiralty_merged.csv'

-> Memproses: 'Ang_Mo_Kio'...
  -> Jumlah kolom setelah digabung: 9
  ✅ Berhasil! Data dengan kolom bersih disimpan di: 'Ang_Mo_Kio_merged.csv'

-> Memproses: 'Bukit_Panjang'...
  -> Jumlah kolom setelah digabung: 9
  ✅ Berhasil! Data dengan kolom bersih disimpan di: 'Bukit_Panjang_merged.csv'

-> Memproses: 'Bukit_Timah'...
  -> Jumlah kolom setelah digabung: 9
  ✅ Berhasil! Data dengan kolom bersih disimpan di: 'Bukit_Timah_merged.csv'

-> Memproses: 'Buona_Vista'...
  -> Jumlah kolom setelah digabung: 9
  ✅ Berhasil! Data dengan kolom bersih disimpan di: 'Buona_Vista_merged.csv'

-> Memproses: 'Changi'...
  -> Jumlah kolom setelah digabung: 9
  ✅ Berhasil! Data dengan kolom bersih disimpan di: 'Changi_merg

In [7]:
admiralty_merged_path = '../data/merged/Admiralty_merged.csv'
admiralty_merged = pd.read_csv(admiralty_merged_path)
admiralty_merged.head()

,date,highest_30_min_rainfall_mm,highest_60_min_rainfall_mm,highest_120_min_rainfall_mm,mean_temperature_c,maximum_temperature_c,minimum_temperature_c,mean_wind_speed_km/h,max_wind_speed_km/h
0,2009-01-01,,,,,,,,
1,2009-01-02,,,,,,,,
2,2009-01-03,,,,,,,,
3,2009-01-04,,,,,,,,
4,2009-01-05,,,,,,,,


In [8]:
admiralty_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5448 entries, 0 to 5447
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   date                         5448 non-null   object
 1   highest_30_min_rainfall_mm   5153 non-null   object
 2   highest_60_min_rainfall_mm   5153 non-null   object
 3   highest_120_min_rainfall_mm  5153 non-null   object
 4   mean_temperature_c           5424 non-null   object
 5   maximum_temperature_c        5424 non-null   object
 6   minimum_temperature_c        5424 non-null   object
 7   mean_wind_speed_km/h         5394 non-null   object
 8   max_wind_speed_km/h          5395 non-null   object
dtypes: object(9)
memory usage: 383.2+ KB


In [ ]:
import pandas as pd
import os
import glob
import re # Diperlukan untuk pembersihan nama kolom

# --- PENGATURAN PATH ---
base_train_path = '../data/raw/Test/'
output_path = '../data/merged/Test/'

# --- FUNGSI PEMBERSIH NAMA KOLOM ---
def clean_column_names(df):
    """
    Membersihkan dan menyeragamkan nama kolom pada DataFrame.
    """
    new_columns = []
    for col in df.columns:
        cleaned_col = col.strip().replace(' ', '_').lower()
        cleaned_col = re.sub(r'[().%°]', '', cleaned_col)
        new_columns.append(cleaned_col)
    df.columns = new_columns
    return df

# --- LOGIKA UTAMA ---
def proses_semua_lokasi(path_induk, path_keluaran):
    """
    Melakukan loop ke semua folder, menggabungkan CSV, membersihkan nama kolom,
    dan menyimpan hasilnya.
    """
    print("Memulai proses otomatis untuk semua lokasi...")
    os.makedirs(path_keluaran, exist_ok=True)
    print(f"Folder output dipastikan ada di: {path_keluaran}")

    try:
        folder_lokasi = [f.name for f in os.scandir(path_induk) if f.is_dir()]
        print(f"Ditemukan {len(folder_lokasi)} folder lokasi untuk diproses.")
    except FileNotFoundError:
        print(f"❌ Error: Direktori induk tidak ditemukan di '{path_induk}'. Hentikan proses.")
        return

    for nama_lokasi in folder_lokasi:
        print(f"\n-> Memproses: '{nama_lokasi}'...")

        path_lokasi_spesifik = os.path.join(path_induk, nama_lokasi)
        daftar_file_csv = sorted(glob.glob(os.path.join(path_lokasi_spesifik, "*.csv")))

        if not daftar_file_csv:
            print(f"  -- Lewati '{nama_lokasi}', tidak ada file .csv ditemukan.")
            continue

        # PENTING: Lakukan pembersihan nama kolom SEBELUM digabung
        list_of_dfs = []
        for file in daftar_file_csv:
            df = pd.read_csv(file)
            # Langsung bersihkan nama kolom setelah file dibaca
            df = clean_column_names(df)
            list_of_dfs.append(df)

        # Gabungkan DataFrame yang nama kolomnya sudah seragam
        df_gabungan = pd.concat(list_of_dfs, ignore_index=True)

        # Cek jumlah kolom setelah digabung
        print(f"  -> Jumlah kolom setelah digabung: {len(df_gabungan.columns)}")

        nama_file_output = f"{nama_lokasi}_gabungan_bersih.csv"
        path_file_output = os.path.join(path_keluaran, nama_file_output)

        df_gabungan.to_csv(path_file_output, index=False)
        print(f"  ✅ Berhasil! Data dengan kolom bersih disimpan di: '{nama_file_output}'")

    print("\n\n🎉 Semua proses selesai!")

# --- JALANKAN PROSES ---
proses_semua_lokasi(base_train_path, output_path)

In [ ]:
bukit_panjang_test = pd.read_csv('../data/merged/Test/Bukit_Panjang_gabungan_bersih.csv')
bukit_panjang_test.info()

In [ ]:
import pandas as pd
import os
import glob
import re
import numpy as np # Import numpy untuk menggunakan np.nan

# --- PENGATURAN PATH ---
# Path utama ke folder Test Anda
base_test_path = '../data/raw/Test/'
# Path dan nama file untuk output final dataset test
output_file = '../data/processed/test.csv'

# --- FUNGSI PEMBERSIH NAMA KOLOM ---
def clean_column_names(df):
    """Membersihkan dan menyeragamkan nama kolom pada DataFrame."""
    new_columns = []
    for col in df.columns:
        cleaned_col = col.strip().replace(' ', '_').lower()
        cleaned_col = re.sub(r'[().%°/]', '', cleaned_col)
        new_columns.append(cleaned_col)
    df.columns = new_columns
    return df

# --- LOGIKA UTAMA ---
def proses_data_test(path_induk, path_output):
    """
    Menggabungkan semua file CSV dari folder Test, membersihkannya,
    dan menambahkan kolom kosong untuk prediksi.
    """
    print("Memulai proses pembuatan dataset test final...")
    try:
        folder_lokasi = [f.name for f in os.scandir(path_induk) if f.is_dir()]
        print(f"Ditemukan {len(folder_lokasi)} folder lokasi untuk diproses.")
    except FileNotFoundError:
        print(f"❌ Error: Direktori Test tidak ditemukan di '{path_induk}'. Hentikan proses.")
        return

    all_locations_data = []
    for nama_lokasi in folder_lokasi:
        print(f"  -> Memproses: '{nama_lokasi}'...")
        path_lokasi_spesifik = os.path.join(path_induk, nama_lokasi)
        daftar_file_csv = sorted(glob.glob(os.path.join(path_lokasi_spesifik, "*.csv")))
        if not daftar_file_csv:
            continue

        list_of_dfs = []
        for file in daftar_file_csv:
            df = pd.read_csv(file)
            df = clean_column_names(df)
            list_of_dfs.append(df)

        df_gabungan_lokasi = pd.concat(list_of_dfs, ignore_index=True)
        df_gabungan_lokasi['lokasi'] = nama_lokasi
        all_locations_data.append(df_gabungan_lokasi)

    if not all_locations_data:
        print("Tidak ada data yang berhasil diproses.")
        return

    final_test_dataset = pd.concat(all_locations_data, ignore_index=True)

    print("\nMenyeragamkan tipe data...")
    kolom_untuk_diubah = [
        'highest_30_min_rainfall_mm', 'highest_60_min_rainfall_mm',
        'highest_120_min_rainfall_mm', 'mean_temperature_c',
        'maximum_temperature_c', 'minimum_temperature_c',
        'mean_wind_speed_km/h', 'max_wind_speed_km/h'
    ]
    for col in kolom_untuk_diubah:
        if col in final_test_dataset.columns:
            final_test_dataset[col] = pd.to_numeric(final_test_dataset[col], errors='coerce')

    final_test_dataset['date'] = pd.to_datetime(final_test_dataset['date'])

    # ==========================================================
    # ===> TAMBAHKAN KOLOM KOSONG UNTUK PREDIKSI <===
    # ==========================================================
    final_test_dataset['daily_rainfall_total_mm'] = np.nan
    print("-> Kolom 'daily_rainfall_total_mm' kosong berhasil ditambahkan.")

    # 4. Simpan dataset final
    final_test_dataset.to_csv(path_output, index=False)

    print("\n\n🎉 Dataset test final berhasil dibuat!")
    print(f"Disimpan di: {path_output}")
    print("\n--- Informasi Dataset Final Test (dengan kolom prediksi) ---")
    final_test_dataset.info()

# --- JALANKAN PROSES ---
proses_data_test(base_test_path, output_file)

In [ ]:
test_merged = pd.read_csv('../data/processed/test.csv')
test_merged.head()

In [ ]:
test_merged['date'] = pd.to_datetime(test_merged['date'])
test_merged['tahun_bulan'] = test_merged['date'].dt.strftime('%Y-%m')
test_merged['tahun_bulan'] = pd.to_datetime(test_merged['tahun_bulan'])
test_merged = pd.merge(test_merged, external_features, on = 'tahun_bulan', how = 'left')

test_merged.drop(columns = ['tahun_bulan'], inplace = True)

In [ ]:
test_merged.info()